In [ ]:
df_themes = df[['DATE', 'Locations', 'V2Themes']].copy()
df_themes['theme'] = df_themes['V2Themes'].apply(extract_themes)
df_themes = df_themes.explode('theme')  # Crea una riga per ogni tema


# Conta i temi unici
theme_counts = df['V2thems'].value_counts().reset_index()
theme_counts.columns = ['theme', 'count']